In [1]:
from deap import base, creator, tools, algorithms
import numpy
import pandas as pd
import random

# Carga los datos del JSON
data = pd.read_json('test.json')

# Define el individuo
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_materia", random.choice, data['Código'].tolist())
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_materia, n=3)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

dias_libres = ["Lunes", "Miércoles", "Jueves", "Viernes"]
preferencia_carga = "Practica"

# Define la función de aptitud
def evaluar(individual):
    score = 0
    troncal_included = False

    # Crea un conjunto para cada día de la semana
    days = {day: set() for day in dias_libres}

    for code in individual:
        # Busca la materia en los datos cargados del JSON
        materia = data[data["Código"] == code]
        if not materia.empty:

            # Verifica si el día de la materia está en los días libres del estudiante
            for dia in materia["Día"].values[0]:
                if dia not in days or code in days[dia]:
                    # Si el día no está en los días libres, devuelve una aptitud muy baja
                    score -= 150
                else:
                    # Si no, añade la materia al conjunto de ese día
                    days[dia].add(code)

            # Verifica si el día de la materia está en los días libres del estudiante
            if any(dia in dias_libres for dia in materia["Día"].values[0]):
                score += 30

            # Aumenta el puntaje según la preferencia de carga del estudiante
            if preferencia_carga == "Practica":
                if materia["Practica"].values[0] == "alta":
                    score += 10
                if materia["Teoria"].values[0] == "alta":
                    score -= 10
            elif preferencia_carga == "Practica":
                if materia["Practica"].values[0] == "alta":
                    score += 10
                if materia["Teoria"].values[0] == "alta":
                    score -= 10

            # Aumenta el puntaje según el puntaje del profesor
            score += (float(materia["Puntaje"].values[0].replace(",", "."))/10)

            # Verifica si la materia es troncal
            if materia["Tipo"].values[0] == "Troncal":
                troncal_included = True

    # Penaliza si no se incluye una materia troncal
    if not troncal_included:
        score -= 50

    return score,

toolbox.register("evaluate", evaluar)

# Define una función para obtener las materias seleccionadas de un individuo
def get_materias(individual):
    materias_seleccionadas = []
    for code in individual:
        materia = data[data["Código"] == code]
        if not materia.empty:
            materias_seleccionadas.append(materia.to_dict('records')[0])
    return materias_seleccionadas

def has_common_day(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    return len(set1.intersection(set2)) > 0

def isValid(individual):
    # Initialize a dictionary to map each day to a list of subject codes
    days = {day: [] for day in dias_libres}
    for subject_code in individual:
        # Convert the subject code to the corresponding subject dictionary
        subject = data[data["Código"] == subject_code].iloc[0].to_dict()
        for day in subject["Día"]:
            # Check if the day is a free day
            if day not in dias_libres:
                return False
            # Check if the day is already occupied
            if days[day]:
                return False
            # If the day is not occupied, schedule the subject on this day
            days[day].append(subject_code)
        # print(days)     
    return True

def mutUnique(individual, indpb):
    for i in range(len(individual)):
        if random.random() < indpb:
            swap_idx = random.randint(0, len(individual) - 1)
            while swap_idx == i:
                swap_idx = random.randint(0, len(individual) - 1)
            
            # Swap the subjects
            individual[i], individual[swap_idx] = individual[swap_idx], individual[i]
            
            # Check if the new combination is valid
            if not isValid(individual):
                # If the new combination is not valid, undo the swap
                individual[i], individual[swap_idx] = individual[swap_idx], individual[i]
    return individual,

def cxTwoPoint(ind1, ind2):
    size = min(len(ind1), len(ind2))
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    if not isValid(ind1) or not isValid(ind2):
        ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] = ind2[cxpoint1:cxpoint2], ind1[cxpoint1:cxpoint2]

    return ind1, ind2

toolbox.register("mate", tools.cxTwoPoint)
# toolbox.register("mutate", tools.mutUniformInt, low=0, up=len(data)-1, indpb=0.1)
toolbox.register("mutate", mutUnique, indpb=0.1)
# toolbox.register("mate", cxUnique)

toolbox.register("select", tools.selTournament, tournsize=3)

def main():
    pop = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.7, mutpb=0.3, ngen=40, stats=stats, halloffame=hof, verbose=True)

    return pop, log, hof

if __name__ == "__main__":
    pop, log, hof = main()
    best_ind = hof[0]
    print("Best individual is %s, %s" % (best_ind, best_ind.fitness.values))
    print("About to call get_materias()")
    materias = get_materias(best_ind)
    print("Finished calling get_materias()")
    print("Materias seleccionadas:")
    for materia in materias:
        print(materia)

gen	nevals	avg     	min    	max    
0  	100   	-39.2918	-312.61	147.702
1  	82    	63.4484 	-272.139	148.027
2  	78    	104.023 	-43.746 	148.027
3  	82    	107.817 	-71.419 	148.358
4  	81    	112.095 	-62.218 	148.358
5  	79    	120.151 	-61.374 	148.358
6  	82    	120.252 	-52.218 	148.358
7  	77    	121.869 	-52.218 	148.358
8  	76    	120.711 	-51.66  	148.358
9  	76    	123.812 	-51.66  	148.358
10 	74    	137.85  	-51.608 	148.358
11 	63    	111.858 	-51.887 	148.358
12 	79    	97.358  	-51.887 	148.358
13 	84    	73.8716 	-51.887 	148.358
14 	73    	97.8852 	-51.608 	148.358
15 	75    	129.366 	-51.608 	148.358
16 	75    	142.358 	-1.921  	148.358
17 	80    	148.358 	148.358 	148.358
18 	89    	148.358 	148.358 	148.358
19 	79    	148.358 	148.358 	148.358
20 	80    	148.358 	148.358 	148.358
21 	79    	148.358 	148.358 	148.358
22 	74    	144.858 	-51.608 	148.358
23 	65    	148.358 	148.358 	148.358
24 	86    	148.358 	148.358 	148.358
25 	88    	148.358 	148.358 	148.358
26 